# Personalized emails with Amazon Personalize and Generative AI - Part 2<a class="anchor" id="top"></a>


## Outline

1. [Introduction](#intro)
1. [Get Personalized recommendations from Amazon Personalize](#getRecs)
1. [Get the user's favorite movie genre](#getGenre)
1. [Add User demographic information](#getDemographic)
1. [Using Amazon Bedrock](#Bedrock)
1. [Generate personalized Marketing Communication](#Emails)
1. [Wrap up](#wrapup)



## Introduction <a class="anchor" id="outline"></a>

In the previous notebook: [`03_Train_Personalize_Model_02_Training.ipynb`](03_Train_Personalize_Model_02_Training.ipynb) you trained and evaluated an Amazon Personalize top-pics-for-you recommender to generate personalize recommendations for each user.

In this Notebook we will get recommendations using the Amazon Personalzie recommender we trainined in the previous notebook. Then we will get metadata for the recommended movies from the catalog and generate Personalized marketing communication for different users experimenting with different prompts and user demographics.


Connect to Amazon Personalize via [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html), the AWS SDK for Python and check the resources have been created.

Similar to the previous notebook, start by importing the relevant packages, and set up a connection to Amazon Personalize using the SDK.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd
import re

In [ ]:
# retrive the saved variables from the previous notebook
%store -r

## Get Personalized recommendations from Amazon Personalize<a class="anchor" id="getRecs"></a>
[Back to top](#top)

Now that we have trained the [Top picks for you](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case) recommender we can get recommendations for our users. 

For more details and ways to use Amazon Personalize to get recommendations, please see the [Amazon Personalize Getting Recommendations](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html).

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

Select a random user to see their recommendations.

In [ ]:
user_id = random.sample(list(user_ids), 1)[0]
user_id

Get 15 recommendations from the 'Top pics for you' recommender we trained.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_top_picks_arn,
    userId = str(user_id),
    numResults = 15
)

print (get_recommendations_response['itemList'])

It works, we can get recommendations!

Let's create a method to get recommended movies and their metadata for each user.

In [ ]:
def getRecommendedMoviesForUserId(
    user_id, 
    workshop_recommender_top_picks_arn, 
    item_data, 
    number_of_movies_to_recommend = 5):
    # For a user_id, gets the top n (number_of_movies_to_recommend) movies using Amazon Personalize 
    # and gets the additional metadata for each movie (item_id) from the item_data.
    # returns a list of movie dictionaries (movie_list) with the relevant data.

    # get recommended movies
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        numResults = number_of_movies_to_recommend
    )

    # create a list of movies with title, geners, and plot.   
    movie_list = []
    
    for movie_id in get_recommendations_response['itemList']:
        
        movie_data = item_data.loc[item_data['ITEM_ID'] == movie_id['itemId']]
        movie_list.append(
            {
                'title' : movie_data['TITLE'].tolist()[0],
                'genres' : movie_data['GENRES'].tolist()[0].replace('|', ' and '),
                'plot' : movie_data['PLOT'].tolist()[0]
            }
        )
    return movie_list
    

We select a random user_id.

In [ ]:
user_id = random.sample(list(user_ids), 1)[0]
user_id

We can select how many movies we want to recommend. In this case 3.

In [ ]:
number_of_movies_to_recommend = 3 

Let's get the recommended movies for the user.

In [ ]:
movie_list = getRecommendedMoviesForUserId(user_id, workshop_recommender_top_picks_arn, item_data, number_of_movies_to_recommend)

# print each movie in the array
for movie in movie_list:
    print ('Title: '+movie['title'])
    print ('Genres: '+movie['genres'])
    print ('Plot: '+movie['plot'])
    print ()

## Get the user's favorite movie genre<a class="anchor" id="getGenre"></a>
[Back to top](#top)

In order to provide a better personalized marketing communication, in this section we calculate the user's favorite movie genre based on the genres of all the movies they have interacted with in the past.

In [ ]:
def getUserFavouriteGenres(user_id, interactions_df, movie_data):
    # For a user_id, gets the users favourite genre by looking at the user's interactions 
    # with each movie in the past and counting the genres to find the most comon genre. 

    # Get all movies the user has watched     
    movies_df = interactions_df[interactions_df['USER_ID'] == user_id]

    genres = {}

    for movie_id in movies_df['ITEM_ID']:

        movie_genres = movie_data[movie_data['ITEM_ID']==movie_id]['GENRES']
        
        if not len(movie_genres.tolist())==0:
            for movie_genre in movie_genres.tolist()[0].split('|'):
                if movie_genre in genres:
                    genres[movie_genre] +=1
                else:
                    genres[movie_genre] = 1

    genres_df = pd.DataFrame(list(genres.items()), columns =['GENRE', 'COUNT'])
    
    # Sort by most common
    genres_df.sort_values(by=['COUNT'], inplace=True, ascending = False)
    
    # Return the most common (favourite) genre       
    return genres_df.iloc[[0]]['GENRE'].values[0]
    

In [ ]:
user_favorite_genre = getUserFavouriteGenres(user_id, interactions_df, item_data)
user_favorite_genre

We get user preferred genres from for this user by counting the number of interactions they have with each genre in the past.

## Add User demographic information<a class="anchor" id="getDemographic"></a>
[Back to top](#top)

We'll generate emails by assuming two different demographics for the users.

Note: Please note that the used version ml-latest-small dataset from the Movielens Dataset (https://grouplens.org/datasets/movielens/) does not contain demographic data and therefore we are trying out multiple options. In a real world scenario, you may know the demographics of your audience.

In [ ]:
# Sample user demographics
user_demographic_1 = f'The user is a 50 year old adult called Otto.'
user_demographic_3 = f'The user is a young adult called Jane.'


## Using Amazon Bedrock<a class="anchor" id="Bedrock"></a>
[Back to top](#top)

[Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html) is a fully managed service that makes base models from Amazon and third-party model providers accessible through an API.

<div class="alert alert-block alert-warning">
<b>Note:</b> Amazon Bedrock users need to request access to models before they are available for use. If you want to add additional models for text, chat, and image generation, you need to request access to models in Amazon Bedrock. To request access to additional models, select the Model access link in the left side navigation panel in the Amazon Bedrock console. For more information see: https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html
</div>


### Connect to Amazon Bedrock
Connect to Amazon Bedrock via [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html), the AWS SDK for Python.

in this example we will be using [Anthropic Claude](https://aws.amazon.com/bedrock/claude/) on Amazon Bedrock.

In [ ]:
bedrock = boto3.client('bedrock-runtime') 

In [ ]:
# Model parameters
# The LLM we will be using
model_id = 'anthropic.claude-v1'

# The desired MIME type of the inference body in the response. 
accept = 'application/json'

# The MIME type of the input data in the request. 
content_type = 'application/json'

# The maximum number of tokens to use in the generated response
max_tokens_to_sample = 1000

## Generate Personalized Marketing Emails<a class="anchor" id="Emails"></a>
[Back to top](#top)


### Generating a marketing email using a simple prompt
Let's generate a simple marketing email that just uses the recommended movies.

In [ ]:
def generate_personalize_simple_prompt(movie_list, model_id, max_tokens_to_sample = 50):

    prompt_template = f'''\n\nHuman: Write a marketing email advertising several movies available in a video-on-demand streaming platform next week, given the movie and user information below. The movies to recommend and their information is contained in the <movie> tag. Put the email between <email> tags.

    <movie>
    {movie_list}
    </movie>

    Assistant: Email body:
    <email>.

    Assistant:
    '''
    
    if 'claude' in model_id:
        prompt_input = json.dumps({"prompt": prompt_template, "max_tokens_to_sample": max_tokens_to_sample })
    
    return prompt_input

In [ ]:
print ('User\'s recommended movies:')

# print each movie in the array
for movie in movie_list:
    print ('Title: '+movie['title'])
    print ('Genres: '+movie['genres'])
    print ('Plot: '+movie['plot'])
    print ()

In [ ]:
# Create prompt input
prompt_input_json = generate_personalize_simple_prompt( movie_list, model_id, max_tokens_to_sample )
prompt_input_json

Let's invoke the model.

In [ ]:
def invoke_llm(bedrock_client, prompt_input_json, model_id, accept, content_type):
    response = bedrock_client.invoke_model(
        body = prompt_input_json, 
        modelId = model_id, 
        accept = accept, 
        contentType = content_type)
    
    response_body = json.loads(response.get('body').read())

    if 'claude' in model_id:
        model_ouput_str_response = response_body['completion']
        
    # Clean Gen AI response
    model_output_str_clean = re.sub(r'<[^>]*>', '', model_ouput_str_response)

    return model_output_str_clean

Next we need to invoke the model. 

<div class="alert alert-block alert-warning">
<b>Note:</b> Amazon Bedrock users need to request access to models before they are available for use. If you get an `Access Denied` error, make sure you have requested access to this model. To request access to additional models, select the Model access link in the left side navigation panel in the Amazon Bedrock console. For more information see: https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html
</div>


In [ ]:
# Invoke model
response = invoke_llm(bedrock, prompt_input_json, model_id, accept, content_type)
print (response)

### Generating a marketing email using a more advanced prompt
In this example we are adding the favorite genre for the user as well as exploring two different possible demographics in the input.

In [ ]:
def generate_personalize_advanced_prompt(user_demographic, favorite_genre, movie_list, model_id, max_tokens_to_sample = 50):

    prompt_template = f'''\n\nHuman: You are a skilled publicist. Write a high-converting marketing email advertising several movies available in a video-on-demand streaming platform next week, 
    given the movie and user information below. Your email will leverage the power of storytelling and persuasive language. 
    You want the email to impress the user, so make it appealing to them based on the information contained in the <user> tags, 
    and take into account the user's favorite genre in the <genre> tags. 
    The movies to recommend and their information is contained in the <movie> tag. 
    All movies in the <movie> tag must be recommended. Give a summary of the movies and why the human should watch them. 
    Put the email between <email> tags.

    <user>
    {user_demographic}
    </user>

    <genre>
    {favorite_genre}
    </genre>

    <movie>
    {movie_list}
    </movie>

    Assistant:
    '''
    
    if 'claude' in model_id:
        prompt_input = json.dumps({"prompt": prompt_template, "max_tokens_to_sample": max_tokens_to_sample })
    
    return prompt_input


### Generating a marketing email for a 50 year old user

In [ ]:
print ('User\'s demographic')
user_demographic = user_demographic_1
user_demographic

In [ ]:
print ('User\'s favorite Genre')
user_favorite_genre

In [ ]:
print ('User\'s recommended movies:')

# print each movie in the array
for movie in movie_list:
    print ('Title: '+movie['title'])
    print ('Genres: '+movie['genres'])
    print ('Plot: '+movie['plot'])
    print ()

In [ ]:
# Create prompt input
prompt_input_json = generate_personalize_advanced_prompt(user_demographic, user_favorite_genre, movie_list, model_id, max_tokens_to_sample )
prompt_input_json

In [ ]:
# Invoke model
response = invoke_llm(bedrock, prompt_input_json, model_id, accept, content_type)
print (response)

### Generating a marketing email for a young adult

In [ ]:
print ('User\'s age group')
user_demographic = user_demographic_3
user_demographic

In [ ]:
print ('User\'s favorite Genre')
user_favorite_genre

In [ ]:
print ('User\'s recommended movies:')

# print each movie in the array
for movie in movie_list:
    print ('Title: '+movie['title'])
    print ('Genres: '+movie['genres'])
    print ('Plot: '+movie['plot'])
    print ()

In [ ]:
prompt_input_json = generate_personalize_advanced_prompt(user_demographic, user_favorite_genre, movie_list, model_id, max_tokens_to_sample )
prompt_input_json

In [ ]:
# Invoke model
response = invoke_llm(bedrock, prompt_input_json, model_id, accept, content_type)
print (response)

## Wrap up<a class="anchor" id="wrapup"></a>
[Back to top](#top)


In [ ]:
# Store variables
%store workshop_dataset_group_arn
%store region
%store role_name

With that you now have a fully working personalized marketing conten generator.

You'll want to make sure that you clean up all of the resources deployed during this workshop. We have provided a separate notebook which shows you how to identify and delete the resources in [`05_Clean_Up.ipynb`](05_Clean_Up.ipynb).